In this notebook we create few plots of wind intensity measured by satellite, with a particular interest on the Canary Islands, which frequently generate the so-called *von Karman* vortex street.

In [12]:
import datetime
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import windplot
import datareading
import logging
import copy
from importlib import reload
reload(windplot)

INFO:root:Starting


<module 'windplot' from '/home/ctroupin/Projects/FNRS-UpwellingFilaments/Filament-Submesoscale/python/windplot.py'>

In [13]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("Starting")

INFO:root:Starting


## Input parameters
We set the date of interest and the domain coordinates are taken from the geoTIFF file.

In [14]:
year, month, day = 2017, 9, 4
figdir = "./figures/"
visibledir = "/data/Wind/visible/"
os.path.isdir(visibledir)

True

## Data

In [15]:
datewind = datetime.datetime(year, month, day)
dayofyear = datewind.timetuple().tm_yday

### Visible image

In [23]:
visibleim = datewind.strftime("viirs2-%Y-%m-%dT00_00_00Z.tiff")
#visibleim = datewind.strftime("Sentinel-3_OLCI_%Y-%m-%d.tiff")
#visibleim = datewind.strftime("Sentinel-2_L1C_%Y-%m-%d.tiff")
#visibleim = datewind.strftime("Terra-%Y-%m-%dT00_00_00Z.tiff")
if os.path.exists(os.path.join(visibledir, visibleim)):
    print("Reading geoTIFF file {}".format(visibleim))
    lonvis, latvis, arr, inproj, extent = datareading.read_geotiff(os.path.join(visibledir, visibleim))
else:
    logger.warn("No visible image, specify domain coordinates")
    

Reading geoTIFF file viirs2-2017-09-04T00_00_00Z.tiff


In [24]:
extent2 = copy.deepcopy(extent)
extent2[0] -= 360.
extent2[1] -= 360.

### Scatterometer wind
The wind (netCDF) are read from the JPL OPEnDAP server.      
The files are sorted by years and days of year.

With these pieces of information we can create two lists of files (one for MetOp-A, one for MetOp-A).

In [25]:
filelist1 = windplot.get_filelist_url(year, dayofyear)

KeyboardInterrupt: 

In [28]:
filename = "ascat_20170904_221800_metopb_25761_eps_o_coa_2401_ovw"
filelistshort = [f for f in filelist1 if filename in f]
filelistshort

['https://opendap.jpl.nasa.gov:443/opendap/OceanWinds/ascat/preview/L2/metop_b/coastal_opt/2017/247/ascat_20170904_221800_metopb_25761_eps_o_coa_2401_ovw.l2.nc.gz']

In [32]:
reload(windplot)
filelistclean = []
for ifile, dataurl in enumerate(filelistshort):

    logger.info("Working on {}".format(dataurl))
    lon, lat, uwind, vwind, windspeed, res = windplot.read_data_domain(dataurl, extent)

    if res:
        filelistclean.append(dataurl)
        mm = str(month).zfill(2)
        dd = str(day).zfill(2)
        
        fname = "".join(("viirs2-", dataurl.split("/")[-1].split(".")[0]))
        logger.info("Saving figure as {}".format(fname))
        figname = os.path.join(figdir, fname)
        windplot.plot_wind_sat(lon, lat, uwind, vwind, windspeed, extent, figname,
                               arr, cmap=plt.cm.hot_r, clim=[5, 15], 
                               date="{}-{}-{}".format(year, mm, dd))

INFO:root:Starting
INFO:root:Working on https://opendap.jpl.nasa.gov:443/opendap/OceanWinds/ascat/preview/L2/metop_b/coastal_opt/2017/247/ascat_20170904_221800_metopb_25761_eps_o_coa_2401_ovw.l2.nc.gz


In [31]:
uwind